# Data Extraction
This section details the process of extracting question data from a PDF file and corresponding answer keys from a TSV file.

## Importing libraries

* Pandas for data manipulation
* NumPy for the NaN datatype
* PyPDF for its ability to read the pfd files, although it requieres post-processing of the extracted text
* sqlite3 to create load the clean data into `bir_warehouse.db`

In [1]:
import pandas as pd
import numpy as np
from pypdf import PdfReader
import sqlite3

## Selecting year and type of exam

This function, `compute_expected_rows`, takes the exam year and topic as input and returns metadata crucial for subsequent data cleaning and loading steps. This metadata includes the expected maximum number of rows in the final DataFrame, the number of questions per exam, and the acronym for the exam type.  The maximum number of rows is calculated based on the number of questions and options per question for each year, following a specific pattern:

*   2024-2021: 210 questions, 4 options each
*   2020-2019: 185 questions, 4 options each
*   2018-2015: 235 questions, 4 options each
*   2014-2012: 235 questions, 5 options each
*   2011-2004: 260 questions, 5 options each

This information is essential for validating data integrity during the ETL process.  For example, the expected row count helps verify that all questions and options have been correctly extracted and transformed. The exam type acronym aids in consistent file naming and data organization.

In [2]:
def compute_expected_rows(year: int, topic: str) -> tuple:
    """Computes and returns metadata about the exam for a given year and topic

    Args:
        year (int): The year of the exam
        topic (str): The topic of the exam

    Returns:
        tuple: Contains:
            - max_rows: The expected maximum number of rows in the DataFrame
            - num_options: The number of questions in the exam
            - exam_type_acronym: The acronym for the exam type

    """
    info_year_dict: dict = {
        2011: [260, 5],
        2014: [235, 5],
        2018: [235, 4],
        2020: [185, 4],
        2021: [210, 4],
    }
    save_name_dict: dict = {
        "BIOLOGÍA": "bir",
        "FARMACIA": "fir",
        "QUÍMICA": "qir",
        "MEDICINA": "mir",
    }
    max_rows: int = 0
    num_options: int = 0
    for target_year, info_list in info_year_dict.items():
        if year <= target_year:
            max_rows = (info_list[0] * info_list[1]) + info_list[0]
            num_options = info_list[1]
            break
    else:
        max_rows = 1050
        num_options = 4
    return max_rows, num_options, save_name_dict[topic]

### Add year of the exam and topic 

In [3]:
year: int = 2021
topic: str = "QUÍMICA" # Select from: BIOLOGÍA, FARMACIA, QUÍMICA, MEDICINA
total_num_rows: int
num_questions: int
exam_acronym: str
total_num_rows, num_questions, exam_acronym = compute_expected_rows(year, topic)
save_format: str = f"../data/clean/clean_{exam_acronym}_{year}.csv"

## Extracting Questions from Pdf file

This section focuses on extracting the textual content of the exam questions from the provided PDF file.

**Path Definition:**

The path to the PDF file is dynamically constructed using the provided `year` and `topic` variables. This ensures flexibility and allows the script to process different exam files without manual path adjustments.

Finally it stores the output in a DataFrame and the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [4]:
path: str = f"../data/raw/type_1/Raw_Cuaderno_{year}_{topic}_0_C.pdf"
with PdfReader(path) as pdf_file:
    full_text: list = []
    for n in range(2, len(pdf_file.pages)):
        page = pdf_file.pages[n]
        text: str = page.extract_text()
        full_text.extend(text.splitlines())
raw_exam_df: pd.DataFrame = pd.DataFrame(full_text, columns=["text"])
raw_exam_df.head(10)

,text
0,
1,- 1 -
2,1. ¿Cómo se denomina al proceso mediante el que
3,se elimina el sulfuro de hidrógeno presente en...
4,"gas natural, produciendo azufre?:"
5,
6,1. Proceso Frasch.
7,2. Proceso Claus.
8,3. Proceso Kroll.
9,4. Proceso de las cámaras de plomo.


## Extracting Answers from .tsv file into a DataFrame

This section focuses on extracting the answer key data from the provided .tsv file and loading it into a Pandas DataFrame.

**Path Definition and Data Loading:**

The path to the .tsv file is dynamically constructed using the `year` and `topic` variables, mirroring the approach used for the PDF file. This ensures consistency and flexibility in processing different exam files. 

The `pd.read_table()` function is then used to read the .tsv file directly into a Pandas DataFrame. Finally, the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [5]:
raw_answers_df: pd.DataFrame = pd.read_table(f"../data/raw/type_1/Raw_Cuaderno_{year}_{topic}_0_C_Respuestas.tsv")
raw_answers_df.head(10)

,V,RC,V.1,RC.1,V.2,RC.2,V.3,RC.3,V.4,RC.4
0,1,2.0,43,3.0,85,3,127,2.0,169,1.0
1,2,4.0,44,4.0,86,2,128,1.0,170,2.0
2,3,1.0,45,2.0,87,4,129,1.0,171,3.0
3,4,3.0,46,3.0,88,1,130,4.0,172,4.0
4,5,1.0,47,4.0,89,3,131,3.0,173,1.0
5,6,2.0,48,2.0,90,4,132,2.0,174,2.0
6,7,2.0,49,3.0,91,4,133,1.0,175,NaN
7,8,4.0,50,2.0,92,2,134,4.0,176,2.0
8,9,2.0,51,4.0,93,1,135,3.0,177,1.0
9,10,1.0,52,2.0,94,3,136,2.0,178,3.0


# Data Transformation

## Exam pdf

### Removing white space

This section focuses on cleaning the raw text extracted from the PDF exam file, preparing it for further processing. This involves removing extraneous whitespace, handling empty lines, and resetting the DataFrame index.

**1. Removing Page Number Artifacts:**

The extracted text may contain page number artifacts, which are removed by filtering out rows containing the string "Página".

In [6]:
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("Página")]
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("Pagina")]
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains(topic)]

footer_list: list = [f"- {npag} -" for npag in range(0,30,1)]

for footer in footer_list:
    raw_exam_df["text"] = raw_exam_df["text"].str.replace(footer, "")

**2. Striping Leading/Trailing Whitespaces**

Leading and trailing whitespace characters are removed from each text entry using the .str.strip() method.

In [7]:
raw_exam_df = raw_exam_df["text"].str.strip()

**3. Handling Empty Lines**

Empty lines, represented as empty strings, are replaced with NaN (Not a Number) values. 

Then rows containing NaN values, representing empty lines, are removed from the DataFrame using .dropna()

In [8]:
raw_exam_df = raw_exam_df.replace("", np.nan)
raw_exam_df = raw_exam_df.dropna()

**4. Reseting DataFrame Index**

The DataFrame index is reset after removing rows, ensuring a contiguous index and dropping the old index.

In [9]:
raw_exam_df = raw_exam_df.reset_index(drop=True)

**5. Displays the first 10 rows of the DataFrame**

In [10]:
raw_exam_df.head(10)

0      1. ¿Cómo se denomina al proceso mediante el que
1    se elimina el sulfuro de hidrógeno presente en el
2                    gas natural, produciendo azufre?:
3                                   1. Proceso Frasch.
4                                    2. Proceso Claus.
5                                    3. Proceso Kroll.
6                  4. Proceso de las cámaras de plomo.
7       2. La forma alotrópica más estable del fósforo
8                                        elemental es:
9                                1. El fósforo blanco.
Name: text, dtype: object

In [11]:
raw_exam_df.tail(10)

1841                               2. Entre 0 y -0.019ºC.
1842                                         3. -0.019ºC.
1843                        4. Entre -0.019ºC y -0.038ºC.
1844    210. La principal fuente d e colesterol extrac...
1845                         para los tejidos humanos es:
1846            1. Las lipoproteínas de muy baja densidad
1847                                              (VLDL).
1848         2. Las lipoproteínas de baja densidad (LDL).
1849      3. Las lipoproteínas de elevada densidad (HDL).
1850                                4. Los quilomicrones.
Name: text, dtype: object

### Joining Truncated Lines

This section addresses the issue of truncated lines and multi-line questions or options within the extracted exam text. A custom function, `process_multi_line_str`, is used to concatenate these lines, ensuring that each question and option is presented as a single, coherent string.

**Function: `process_multi_line_str(df)`**

This function iterates through the DataFrame, performing two main tasks:

1.  **Joining Truncated Words:**
    * It checks if a line ends with a hyphen ("-"), indicating a word that has been truncated at the end of a line.
    * If a line is truncated, it removes the hyphen and appends the content of the following line to the current line.
    * The following line is then removed from the DataFrame, and the index is reset.

2.  **Joining Multi-Line Questions/Options:**
    * It checks if the first character of the next line can be converted to an integer. This is used as a heuristic to identify the start of a new question or option.
    * If the next line does not start with an integer (i.e., it's a continuation of the current question or option), it is appended to the current line, separated by a space.
    * The next line is then removed, and the index is reset.

In [12]:
def process_multi_line_str(df: pd.DataFrame) -> pd.DataFrame:
    """Function to handle truncated lines

    Args:
        df (pd.DataFrame): A pandas dataframe

    Returns:
        pd.DataFrame: Returns the DataFrame without truncated lines
    """
    i: int = 0
    while i < len(df) - 1:
        if i < len(df) -1  and df.iloc[i].endswith("-"):
            df.iloc[i] = df.iloc[i][:-1] + df.iloc[i + 1]
            df = df.drop(i + 1)
            df = df.reset_index(drop=True)
        else:
            i += 1
    n: int = 0
    while n < len(df) -1:
        if n + 1 < len(df):
            try:
                int(df.iloc[n + 1][0:1])
                n += 1
            except ValueError:
                df.iloc[n] = df.iloc[n] + " " + df.iloc[n + 1]
                df = df.drop(n +1)
                df = df.reset_index(drop= True)
        else:
            break
    return df

The process_multi_line_str function is applied to the cleaned raw_exam_df DataFrame, and the resulting DataFrame is stored in exam_df_concat. The first few rows of the concatenated DataFrame are displayed to verify the results.

In [13]:
exam_df_concat: pd.DataFrame = process_multi_line_str(raw_exam_df)
exam_df_concat.head(10)

0    1. ¿Cómo se denomina al proceso mediante el qu...
1                                   1. Proceso Frasch.
2                                    2. Proceso Claus.
3                                    3. Proceso Kroll.
4                  4. Proceso de las cámaras de plomo.
5    2. La forma alotrópica más estable del fósforo...
6                                1. El fósforo blanco.
7                                  2. El fósforo rojo.
8                               3. El fósforo violeta.
9                                 4. El fósforo negro.
Name: text, dtype: object

### Validating and Correction Incorrect Rows

This section focuses on validating the number of rows in the cleaned DataFrame against the expected number and correcting any rows that do not conform to the expected format.

**1. Calculating Expected and Actual Incorrect Rows:**

We calculate the expected number of incorrect rows by subtracting the total expected rows from the current number of rows in the DataFrame. We then identify the actual number of incorrect rows by counting the rows that do not end with a period ("."), a colon (":") or interrogation sign ("?").

In [14]:
num_rows_incorrect_expected: int = len(exam_df_concat) - total_num_rows
num_rows_incorrect: int = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].count()
id_rows_incorrect: list = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].index.to_list()
print(f"Number of expected incorrect rows = {num_rows_incorrect_expected}")
print(f"Number of rows not ending with '.', ':' or '?' = {num_rows_incorrect}")
print(id_rows_incorrect)

Number of expected incorrect rows = 13
Number of rows not ending with '.', ':' or '?' = 15
[80, 114, 186, 187, 203, 256, 259, 360, 470, 507, 508, 712, 1007, 1027, 1030]


**2. Function: print_rows_incorrrect(wrong_id, df):**

This function takes a list of incorrect row indices and the DataFrame as input. It then prints the row identified as incorrect along with the 2 rows before and 2 rows after, providing context to help determine how to fix the error.

In [15]:
def print_rows_incorrrect(wrong_id: list[int], df: pd.DataFrame) -> None:
    """Print +/- 2 rows for context

    Args:
        wrong_id (list): Generated list of wrong row_id
        df (pd.DataFrame): DataFrame corresponding with the worng_id list
    """
    for wid in wrong_id:
        print(f"Id to fix {wid}:")
        print(df.iloc[wid -2: wid +3])

**3. Function: fix_incorrect(num_row, df):**

This function takes a list of row indices to fix and the DataFrame. It iterates through the indices in reverse order to avoid index shifting problems. Each incorrect row is concatenated with the row immediately following it, and the subsequent row is then dropped. The DataFrame index is reset after each concatenation.

In [16]:
def fix_incorrect(num_row: list[int], df: pd.DataFrame) -> pd.DataFrame:
    """Fix the incorrect rows in the DataFrame

    Args:
        num_row (list): List of ids to fix
        df (pd.DataFrame): The DataFrame to fix

    Returns:
        pd.DataFrame: Fixed
    """
    num_row: list = sorted(num_row, reverse=True)
    for n in num_row:
        df.iloc[n] = df.iloc[n] + df.iloc[n + 1]
        df = df.drop(n + 1)
        df = df.reset_index(drop= True)
    return df

**4. Validation and Correction Logic:**

We compare the expected and actual number of incorrect rows. If they match, we proceed to fix the incorrect rows using the `fix_incorrect` function. Otherwise, we print the context of the incorrect rows and raise a warning, indicating that manual inspection and correction are required.

In [17]:
if num_rows_incorrect_expected == num_rows_incorrect:
    exam_df_fixed = fix_incorrect(id_rows_incorrect, exam_df_concat)
    df_correct_rows: bool = len(exam_df_fixed) == num_rows_incorrect_expected
else:
    print_rows_incorrrect(id_rows_incorrect, exam_df_concat)
    # raise Warning("More incorrect rows that expected, uncomment the lines below and add ids to the list, and comment this line")
    rows_fix: list[int] = [80, 186, 187, 203, 256, 259, 470, 507, 508, 712, 1007, 1027, 1030]
    exam_df_fixed: pd.DataFrame = fix_incorrect(rows_fix, exam_df_concat)

Id to fix 80:
78                                 3. Con el wolframio.
79                                     4. Con el plomo.
80    17. ¿Cuál de las siguientes reacciones es la q...
81                                             200º C?:
82                            1. NH4ClO4 → NH3 + HClO4.
Name: text, dtype: object
Id to fix 114:
112          1. Tetraedros [SiO4] compartiendo vértices.
113           2. Tetraedros [SiO4] compartiendo aristas.
114            3. Octaedros [SiO6] compartiendo vértices
115            4. Octaedros [SiO6] compartiendo aristas.
116    24. Indica el principal producto de la combust...
Name: text, dtype: object
Id to fix 186:
184    3. Es un gas muy estable a cualquier temperatura.
185    4. Puede utilizarse para aumentar la potencia ...
186                                          38. El 𝑷𝒐𝟖𝟒
187                    𝟐𝟏𝟎  decae espontáneamente a 𝑷𝒃𝟖𝟐
188                               𝟐𝟎𝟔  c o n emisión de:
Name: text, dtype: object
Id to fix 187:
185    4. Pue

**5. Verification of Corrected Rows:**

We print the expected total number of rows and the actual number of rows in the corrected DataFrame to verify the results of the correction process.

In [18]:
print(f"Correct number of rows = {total_num_rows}")
print(exam_df_fixed.shape[0])

Correct number of rows = 1050
1050


### Pivoting the DataFrame

This section focuses on transforming the cleaned exam data into a structured format suitable for analysis. The DataFrame is pivoted to create columns for each question and its corresponding options.

**1. Preparing the DataFrame for Pivoting:**

The `exam_df_fixed` Series is converted to a DataFrame, and a 'group' column is created to identify each question and its options. An 'option_num' column is then generated to enumerate the options within each group.


In [19]:
exam_df_fixed = exam_df_fixed.to_frame()
groups: pd.Series = pd.Series((exam_df_fixed.index // (num_questions+1)) +1)
exam_df_fixed["group"] = groups  
exam_df_fixed["option_num"] = exam_df_fixed.groupby("group").cumcount() + 1

**2. Pivoting the DataFrame:**

The DataFrame is pivoted using the 'group' column as the index, the 'option_num' column as the columns, and the 'text' column as the values. The index is then reset.

In [20]:
exam_df_pivot: pd.DataFrame = exam_df_fixed.pivot(index="group", columns="option_num", values="text")
exam_df_pivot = exam_df_pivot.reset_index()

**3. Renaming Columns:**

The columns are renamed to more descriptive names, such as "Question", "Option_1", "Option_2", etc.

In [21]:
key_list: list = [x for x in range(1, num_questions+2, 1)]
val_list: list = ["Question"] + [f"Option_{x}" for x in range(1, 4 + 1, 1)]

exam_df_pivot = exam_df_pivot.rename_axis(None, axis=1).rename(columns=dict(zip(key_list, val_list)))

**4. Dropping the 'group' Column:**

The 'group' column is dropped as it is no longer needed

In [22]:
exam_df_pivot = exam_df_pivot.drop(columns=["group"])

**5. Displaying the Pivoted DataFrame:**

The first 10 rows of the pivoted DataFrame are displayed to verify the results of the transformation.

In [23]:
exam_df_pivot.head(10)

,Question,Option_1,Option_2,Option_3,Option_4
0,1. ¿Cómo se denomina al proceso mediante el qu...,1. Proceso Frasch.,2. Proceso Claus.,3. Proceso Kroll.,4. Proceso de las cámaras de plomo.
1,2. La forma alotrópica más estable del fósforo...,1. El fósforo blanco.,2. El fósforo rojo.,3. El fósforo violeta.,4. El fósforo negro.
2,3. ¿Es posible obtener algún compuesto en esta...,"1. Si, por ejemplo, el OsO4 y el RuO4.","2. No, pero se puede alcanzar estado de oxidac...","3. No, pero se puede alcanzar estado de oxidac...","4. No, pero se puede alcanzar estado de oxidac..."
3,4. ¿Cuál de las siguientes afirmaciones sobre ...,1. Todos los compuestos iónicos son sólidos y ...,2. Tienen puntos de fusión elevados.,"3. Si no descomponen al calentarlos, en estado...",4. Casi todos se disuelven en disolventes muy ...
4,5. ¿Con qué elementos forma compuestos el flúor?:,"1. Con todos menos con He, Ne y Ar.",2. Con todos menos con los gases nobles.,3. Con todos menos con los alcalinos.,4. Con todos menos con los otros halógenos.
5,6. ¿Qué es la fibra óptica?:,1. Fibra delgada de óxido de hierro de ultra a...,2. Fibra delgada de sílice de ultra alta pureza.,3. Fibra delgada de óxido de hierro con impure...,4. Fibra delgada de sílice con impurezas de óx...
6,7. El anión tiosulfato se empleó de forma muy ...,1. S4O62-.,2. S2O32-.,3. HSO3-.,4. S2O72-.
7,8. El cobalto:,1. Es un metal blando de color metálico.,2. En su forma metálica no tiene propiedades m...,3. Es muy reactivo como elemento químico.,4. Es un elemento esencial para los seres vivos.
8,9. ¿Qué tipo de defecto es un defecto de Schot...,1. Defecto puntual extrínseco.,2. Defecto puntual intrínseco.,3. Defecto extendido.,4. Defecto lineal.
9,10. ¿Cuál de las siguientes reacciones necesit...,1. H2O2 (liq.) → H2O (liq.) + ½ O2 (gas).,2. H2O2 (liq.) + SO2 (gas) → H2SO4.,3. NO (gas) + ½ O2 (gas) → NO2 (gas).,4. Cl2 + 2 NaOH → NaCl + NaClO + H2O.


In [24]:
exam_df_pivot.tail(10)

,Question,Option_1,Option_2,Option_3,Option_4
200,201. ¿Cuál de las siguientes es una metaloenzi...,1. Carboxipeptidasa A.,2. Formilmetanofurano deshidrogenasa.,3. Monóxido de carbono deshidrogenasa.,4. Carboxipeptidas B.
201,202. Las lipoproteínas más ligeras (densidad <...,1. Las de baja densidad (LDL).,2. Las de alta densidad (HDL).,3. Los quilomicrones.,4. Las de muy baja densidad (VLDL).
202,203. ¿Cuántos átomos de nitrógeno contiene una...,1. Cinco.,2. Cuatro.,3. Tres.,4. Seis.
203,204. ¿Cuáles son las configuraciones electróni...,1. Cr: [Ar] 4s 23d5; Cr2+: [Ar] 4s 23d3; Cr3+:...,2. Cr: [Ar] 4s23d5; Cr2+: [Ar] 3d5; Cr3+: [Ar]...,3. Cr: [Ar] 4s 23d4; Cr2+: [Ar] 4s 23d2; Cr3+:...,4. Cr: [Ar] 4s23d4; Cr2+: [Ar] 3d4; Cr3+: [Ar]...
204,"205. En un procedimiento de medida, la menor c...",1. Límite de detección.,2. Límite de cuantificación.,3. Capacidad del procedimiento de medida.,4. Límite de dilución.
205,206. ¿Cuál es la composición del núcleo del de...,1. Dos protones.,2. Dos neutrones.,3. Un protón y un neutrón.,4. Un protón y dos neutrones.
206,207. La Rifampicina inhibe la transcripción en...,1. Se une a la subunidad beta de las RNA polim...,2. Se une en el DNA a las secuencias promotora...,3. Desestabiliza el heteroduplex DNA-RNA forma...,4. Se intercala en el molde de DNA e impide la...
207,208. Si una reacción compuesta por varias etap...,1. EA=-EA1+EA-1-EA2.,2. EA=EA1-EA-1+EA2.,3. EA=-EA1+EA-1+EA2.,4. EA=-EA1-EA-1-EA2.
208,"209. Una disolución de urea en agua, con conce...",1. 0ºC.,2. Entre 0 y -0.019ºC.,3. -0.019ºC.,4. Entre -0.019ºC y -0.038ºC.
209,210. La principal fuente d e colesterol extrac...,1. Las lipoproteínas de muy baja densidad (VLDL).,2. Las lipoproteínas de baja densidad (LDL).,3. Las lipoproteínas de elevada densidad (HDL).,4. Los quilomicrones.


## Answers tsv

This section focuses on cleaning and consolidating the answer key data from the TSV file. The original data contains duplicate columns, which are processed and combined into a single, clean DataFrame.

**1. Defining Column Lists:**

We define lists containing the original column names and the duplicate column names.

In [25]:
original_col: list[str] = ["V", "RC"]
duplicate_cols: list[str] = ["V.1", "RC.1", "V.2", "RC.2", "V.3", "RC.3", "V.4", "RC.4"]

**2. Processing Duplicate Columns:**

We iterate through the duplicate_cols list in pairs, extracting the corresponding columns, renaming them to the original column names, and appending them to a list of DataFrames.

In [26]:
answers_df_list: list[pd.DataFrame] = [raw_answers_df[original_col]]

for i in range(0, len(duplicate_cols), 2):
    pair_cols: list = duplicate_cols[i:i+2]
    df_pair_col: pd.DataFrame = raw_answers_df[pair_cols].rename(columns={pair_cols[0]: "V", pair_cols[1]: "RC"})
    answers_df_list.append(df_pair_col)

**3. Concatenating DataFrames:**

The list of DataFrames is concatenated into a single DataFrame, ignoring the original index.

In [27]:
answers_df_clean: pd.DataFrame = pd.concat(answers_df_list, ignore_index=True)

**4. Displaying the Cleaned DataFrame:**

The cleaned DataFrame is printed to verify the results of the consolidation process.

In [28]:
answers_df_clean.head(10)

,V,RC
0,1,2.0
1,2,4.0
2,3,1.0
3,4,3.0
4,5,1.0
5,6,2.0
6,7,2.0
7,8,4.0
8,9,2.0
9,10,1.0


## Joining the DataFrames

This section focuses on merging the processed exam questions DataFrame with the cleaned answers DataFrame, performing final data cleaning, and verifying the integrity of the joined data.

**1. Joining the DataFrames:**

The pivoted exam questions DataFrame (`exam_df_pivot`) and the cleaned answers DataFrame (`answers_df_clean`) are joined horizontally using `pd.concat`. The 'V0' column, which represents the question number from the answers DataFrame, is then dropped. The first 10 rows of the joined DataFrame are displayed.

In [29]:
processed_exam_df: pd.DataFrame = pd.concat([exam_df_pivot, answers_df_clean], axis=1)
processed_exam_df = processed_exam_df.drop(columns=["V"])
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC
0,1. ¿Cómo se denomina al proceso mediante el qu...,1. Proceso Frasch.,2. Proceso Claus.,3. Proceso Kroll.,4. Proceso de las cámaras de plomo.,2.0
1,2. La forma alotrópica más estable del fósforo...,1. El fósforo blanco.,2. El fósforo rojo.,3. El fósforo violeta.,4. El fósforo negro.,4.0
2,3. ¿Es posible obtener algún compuesto en esta...,"1. Si, por ejemplo, el OsO4 y el RuO4.","2. No, pero se puede alcanzar estado de oxidac...","3. No, pero se puede alcanzar estado de oxidac...","4. No, pero se puede alcanzar estado de oxidac...",1.0
3,4. ¿Cuál de las siguientes afirmaciones sobre ...,1. Todos los compuestos iónicos son sólidos y ...,2. Tienen puntos de fusión elevados.,"3. Si no descomponen al calentarlos, en estado...",4. Casi todos se disuelven en disolventes muy ...,3.0
4,5. ¿Con qué elementos forma compuestos el flúor?:,"1. Con todos menos con He, Ne y Ar.",2. Con todos menos con los gases nobles.,3. Con todos menos con los alcalinos.,4. Con todos menos con los otros halógenos.,1.0
5,6. ¿Qué es la fibra óptica?:,1. Fibra delgada de óxido de hierro de ultra a...,2. Fibra delgada de sílice de ultra alta pureza.,3. Fibra delgada de óxido de hierro con impure...,4. Fibra delgada de sílice con impurezas de óx...,2.0
6,7. El anión tiosulfato se empleó de forma muy ...,1. S4O62-.,2. S2O32-.,3. HSO3-.,4. S2O72-.,2.0
7,8. El cobalto:,1. Es un metal blando de color metálico.,2. En su forma metálica no tiene propiedades m...,3. Es muy reactivo como elemento químico.,4. Es un elemento esencial para los seres vivos.,4.0
8,9. ¿Qué tipo de defecto es un defecto de Schot...,1. Defecto puntual extrínseco.,2. Defecto puntual intrínseco.,3. Defecto extendido.,4. Defecto lineal.,2.0
9,10. ¿Cuál de las siguientes reacciones necesit...,1. H2O2 (liq.) → H2O (liq.) + ½ O2 (gas).,2. H2O2 (liq.) + SO2 (gas) → H2SO4.,3. NO (gas) + ½ O2 (gas) → NO2 (gas).,4. Cl2 + 2 NaOH → NaCl + NaClO + H2O.,1.0


**2. Identifying and Handling Missing Values:**

Missing values are then replaced with 0.

In [30]:
processed_exam_df = processed_exam_df.fillna(0)

**3. Verifying Missing Values and Data Types:**

The number of missing values in each column is checked, and the data types of the columns are displayed. The 'RC' (correct answer) column is then converted to integer type.

In [31]:
processed_exam_df.isnull().sum()
processed_exam_df["RC"] = processed_exam_df["RC"].astype(int)
processed_exam_df.dtypes

Question    object
Option_1    object
Option_2    object
Option_3    object
Option_4    object
RC           int32
dtype: object

**4. Adding year column**

A column containing the year of the exam is added to the DataFrame.

In [32]:
processed_exam_df["year"] = year

**5. Displaying the Final DataFrame:**

The first 10 rows of the final joined and cleaned DataFrame are displayed.

In [33]:
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
0,1. ¿Cómo se denomina al proceso mediante el qu...,1. Proceso Frasch.,2. Proceso Claus.,3. Proceso Kroll.,4. Proceso de las cámaras de plomo.,2,2021
1,2. La forma alotrópica más estable del fósforo...,1. El fósforo blanco.,2. El fósforo rojo.,3. El fósforo violeta.,4. El fósforo negro.,4,2021
2,3. ¿Es posible obtener algún compuesto en esta...,"1. Si, por ejemplo, el OsO4 y el RuO4.","2. No, pero se puede alcanzar estado de oxidac...","3. No, pero se puede alcanzar estado de oxidac...","4. No, pero se puede alcanzar estado de oxidac...",1,2021
3,4. ¿Cuál de las siguientes afirmaciones sobre ...,1. Todos los compuestos iónicos son sólidos y ...,2. Tienen puntos de fusión elevados.,"3. Si no descomponen al calentarlos, en estado...",4. Casi todos se disuelven en disolventes muy ...,3,2021
4,5. ¿Con qué elementos forma compuestos el flúor?:,"1. Con todos menos con He, Ne y Ar.",2. Con todos menos con los gases nobles.,3. Con todos menos con los alcalinos.,4. Con todos menos con los otros halógenos.,1,2021
5,6. ¿Qué es la fibra óptica?:,1. Fibra delgada de óxido de hierro de ultra a...,2. Fibra delgada de sílice de ultra alta pureza.,3. Fibra delgada de óxido de hierro con impure...,4. Fibra delgada de sílice con impurezas de óx...,2,2021
6,7. El anión tiosulfato se empleó de forma muy ...,1. S4O62-.,2. S2O32-.,3. HSO3-.,4. S2O72-.,2,2021
7,8. El cobalto:,1. Es un metal blando de color metálico.,2. En su forma metálica no tiene propiedades m...,3. Es muy reactivo como elemento químico.,4. Es un elemento esencial para los seres vivos.,4,2021
8,9. ¿Qué tipo de defecto es un defecto de Schot...,1. Defecto puntual extrínseco.,2. Defecto puntual intrínseco.,3. Defecto extendido.,4. Defecto lineal.,2,2021
9,10. ¿Cuál de las siguientes reacciones necesit...,1. H2O2 (liq.) → H2O (liq.) + ½ O2 (gas).,2. H2O2 (liq.) + SO2 (gas) → H2SO4.,3. NO (gas) + ½ O2 (gas) → NO2 (gas).,4. Cl2 + 2 NaOH → NaCl + NaClO + H2O.,1,2021


In [34]:
processed_exam_df.tail(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
200,201. ¿Cuál de las siguientes es una metaloenzi...,1. Carboxipeptidasa A.,2. Formilmetanofurano deshidrogenasa.,3. Monóxido de carbono deshidrogenasa.,4. Carboxipeptidas B.,2,2021
201,202. Las lipoproteínas más ligeras (densidad <...,1. Las de baja densidad (LDL).,2. Las de alta densidad (HDL).,3. Los quilomicrones.,4. Las de muy baja densidad (VLDL).,3,2021
202,203. ¿Cuántos átomos de nitrógeno contiene una...,1. Cinco.,2. Cuatro.,3. Tres.,4. Seis.,1,2021
203,204. ¿Cuáles son las configuraciones electróni...,1. Cr: [Ar] 4s 23d5; Cr2+: [Ar] 4s 23d3; Cr3+:...,2. Cr: [Ar] 4s23d5; Cr2+: [Ar] 3d5; Cr3+: [Ar]...,3. Cr: [Ar] 4s 23d4; Cr2+: [Ar] 4s 23d2; Cr3+:...,4. Cr: [Ar] 4s23d4; Cr2+: [Ar] 3d4; Cr3+: [Ar]...,0,2021
204,"205. En un procedimiento de medida, la menor c...",1. Límite de detección.,2. Límite de cuantificación.,3. Capacidad del procedimiento de medida.,4. Límite de dilución.,2,2021
205,206. ¿Cuál es la composición del núcleo del de...,1. Dos protones.,2. Dos neutrones.,3. Un protón y un neutrón.,4. Un protón y dos neutrones.,3,2021
206,207. La Rifampicina inhibe la transcripción en...,1. Se une a la subunidad beta de las RNA polim...,2. Se une en el DNA a las secuencias promotora...,3. Desestabiliza el heteroduplex DNA-RNA forma...,4. Se intercala en el molde de DNA e impide la...,1,2021
207,208. Si una reacción compuesta por varias etap...,1. EA=-EA1+EA-1-EA2.,2. EA=EA1-EA-1+EA2.,3. EA=-EA1+EA-1+EA2.,4. EA=-EA1-EA-1-EA2.,2,2021
208,"209. Una disolución de urea en agua, con conce...",1. 0ºC.,2. Entre 0 y -0.019ºC.,3. -0.019ºC.,4. Entre -0.019ºC y -0.038ºC.,4,2021
209,210. La principal fuente d e colesterol extrac...,1. Las lipoproteínas de muy baja densidad (VLDL).,2. Las lipoproteínas de baja densidad (LDL).,3. Las lipoproteínas de elevada densidad (HDL).,4. Los quilomicrones.,2,2021


### Filtering by relevant questions

**1. Filtering for id**

A list of the number of question is used to filter the DataFrame, but first doing a -1 so it matches with the id. 

Finally the indexes are reseted

In [35]:
id_filter = [x for x in range(121, 160, 1)] + [206, 209]
processed_exam_df_filtered = processed_exam_df.iloc[id_filter]
processed_exam_df_filtered = processed_exam_df_filtered.reset_index(drop= True)
print(f"Expected number of rows = {len(id_filter)}\nNumber of rows in filtered df = {processed_exam_df_filtered.shape[0]}")

Expected number of rows = 41
Number of rows in filtered df = 41


**2. Showing the filtered DataFrame**

Using head and tail

In [36]:
processed_exam_df_filtered.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
0,122. La presencia de cuál de las siguientes di...,1. Dedo de zinc.,2. Hélice α.,3. Cadena β.,4. Región desordenada.,1,2021
1,123. La síntesis de la urea requiere la siguie...,1. Carbamil fosfato sintetasa I.,2. Carbamil fosfato sintetasa II.,3. Ornitina transcarbamilasa.,4. Argininosuccinato sintetasa.,2,2021
2,124. ¿Cuál de los siguientes parámetros se ele...,1. Lactato deshidrogenasa.,2. Potasio.,3. Fosfato inorgánico.,4. Calcio.,1,2021
3,125. Las endonucleasas de restricción (o enzim...,1. Hidrolizan enlaces fos foéster comenzando p...,2. Se utilizan para la formación de moléculas ...,3. Reconocen y cortan el DNA en secuencias lib...,4. Reconocen y cortan el DNA en secuencias de ...,4,2021
4,126. ¿Cuál de las siguientes afirmaciones sobr...,1. Las bases G y C se unen mediante 3 puentes ...,2. Las bases A y T se unen mediante 3 puentes ...,3. Las bases A y G se unen mediante 2 puentes ...,4. Las bases A y C se unen mediante 2 puentes ...,1,2021
5,127. ¿Cuál de estas frases referidas a la teor...,1. La ATP sintasa no se afecta por el gradient...,2. La energía liberada por el transporte elect...,3. La fosforilación oxidativa puede tener luga...,4. Independientemente del punto de entrada de ...,2,2021
6,128. La principal enzima reguladora de la velo...,1. Fosfofructoquinasa1.,2. Hexoquinasa.,3. Piruvato deshidrogenasa.,4. Enolasa.,1,2021
7,129. ¿Cuál es la principal utilidad de la hemo...,1. Valorar el control metabólico de un diabético.,2. Diagnóstico de la diabetes mellitus.,3. Indicar cuando existen complicaciones en un...,4. Indicar cuando puede existir una diabetes g...,1,2021
8,130. El cerebro está adapta do a usar preferen...,1. De alta afinidad y por tanto con elevada Km.,2. De baja afinidad y por tanto con elevada Km.,"3. De alta afinidad, independientemente de su Km.",4. De alta afinidad y por tanto con baja Km.,4,2021
9,131. La acetil-Coenzima A (Acetil-CoA) es port...,1. Grupos acetonitrilo en algunas reacciones e...,2. Grupos amino en algunas reacciones de trans...,3. Grupos acetilo en algunas reacciones enzimá...,4. Grupos etilo en algunas reacciones enzimáti...,3,2021


In [37]:
processed_exam_df_filtered.tail(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
31,153. El escorbuto es una e nfermedad provocada...,1. La ausencia de síntesis de retinal y la con...,2. Un menor rendimiento d el transporte electr...,3. Un defecto en una enzima responsable de la ...,4. La imposibilidad de hidroxilación de la pro...,4,2021
32,154. ¿En cuál de los siguien tes tejidos es de...,1. Adiposo.,2. Cerebral.,3. Hepático.,4. Eritrocítico.,1,2021
33,155. La actividad de corrección de pruebas de ...,1. Polimerizante 5´-3´.,2. Transesterificadora 5- 3´.,3. Exonucleasa 5´ -3´.,4. Exonucleasa 3´ -5´.,4,2021
34,156. ¿Qué ocurrirá en el caso de que el pH san...,1. Se verá afectada la unión de oxígeno a la m...,2. Descenderá la afinidad de la hemoglobina po...,3. Aumentará la afinidad de la hemoglobina por...,"4. La hemoglobina no podrá fijar 2,3bisfosfogl...",2,2021
35,157. La gluconeogénesis se realiza en:,1. Únicamente hígado y corteza adrenal.,"2. Hígado y corteza adrenal y, en condiciones ...","3. Hígado y corteza adrenal y, en condiciones ...",4. Hígado e intestino delgado.,2,2021
36,158. Todos los aminoácidos siguientes contiene...,1. Leucina.,2. Alanina.,3. Isoleucina.,4. Lisina.,4,2021
37,159. ¿Cuál de las siguientes afirmaciones sob...,"1. Es una ruta cerrada, siempre transcurre con...",2. Todas las reacciones de la vía transcurren ...,3. Todas las enzimas implicadas en eta vía son...,4. Algunas de las enzimas implicadas no son ex...,1,2021
38,160. Con relación a la reacción en cadena de l...,"1. Es una reacción química, in vivo, que inhib...",2. Utiliza una mezcla equimolar de ribonucleót...,3. Se hace a través de la acción de una polime...,4. No necesita cebadores.,3,2021
39,207. La Rifampicina inhibe la transcripción en...,1. Se une a la subunidad beta de las RNA polim...,2. Se une en el DNA a las secuencias promotora...,3. Desestabiliza el heteroduplex DNA-RNA forma...,4. Se intercala en el molde de DNA e impide la...,1,2021
40,210. La principal fuente d e colesterol extrac...,1. Las lipoproteínas de muy baja densidad (VLDL).,2. Las lipoproteínas de baja densidad (LDL).,3. Las lipoproteínas de elevada densidad (HDL).,4. Los quilomicrones.,2,2021


# Data Loading

## Saving in csv file

The `processed_exam_df_filtered` DataFrame, containing the cleaned and transformed exam data, is saved to a CSV file with the name generatd in the first step in the "data" directory. The `index=False` argument ensures that the DataFrame index is not written to the file, resulting in a cleaner output. This CSV file can be used for further analysis or as input for other applications.

In [38]:
processed_exam_df_filtered.to_csv(save_format, index= False)

## Writing into data warehouse

This section details the process of loading the cleaned and transformed exam data into an SQLite database, serving as a data warehouse for further analysis and querying.

**1. Database Connection and Path:**

The path to the SQLite database file is defined, and a connection is established using the `sqlite3` library.

In [39]:
db_path: str = "../data/clean/bir_warehouse.db"

**2. SQL Queries Definition:**

Two SQL queries are defined: one for inserting questions into the questions table and another for inserting options into the questions_options table.

In [40]:
query_question: str = """
INSERT INTO questions (exam_year, exam_subject, question) 
VALUES((SELECT id_year FROM year WHERE year_name = ?),
    (SELECT id_type FROM exam WHERE exam_type = ?),
    ?);
"""

query_options: str = """
INSERT INTO questions_options (question_id, option_num, option_text, is_correct)
VALUES((SELECT q.id 
        FROM questions AS q
        JOIN year AS y ON q.exam_year = y.id_year
        WHERE q.question = ? AND y.year_name = ?),
    ?,
    ?,
    ?)
"""


**3. Database Interaction and Data Insertion:**

A with statement is used to establish a connection to the SQLite database, ensuring that the connection is properly closed after use. A cursor is created, and the processed_exam_df DataFrame is iterated over to insert each question and its options

Fixed spacing issue manually in the csv file, so the DataFrame it will be loaded from the csv file

In [42]:
processed_exam_df_filtered = pd.read_csv("../data/clean/clean_qir_2021.csv")
processed_exam_df_filtered.tail()

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
36,158. Todos los aminoácidos siguientes contiene...,1. Leucina.,2. Alanina.,3. Isoleucina.,4. Lisina.,4,2021
37,159. ¿Cuál de las siguientes afirmaciones sob...,"1. Es una ruta cerrada, siempre transcurre con...",2. Todas las reacciones de la vía transcurren ...,3. Todas las enzimas implicadas en eta vía son...,4. Algunas de las enzimas implicadas no son ex...,1,2021
38,160. Con relación a la reacción en cadena de l...,"1. Es una reacción química, in vivo, que inhib...",2. Utiliza una mezcla equimolar de ribonucleót...,3. Se hace a través de la acción de una polime...,4. No necesita cebadores.,3,2021
39,207. La Rifampicina inhibe la transcripción en...,1. Se une a la subunidad beta de las RNA polim...,2. Se une en el DNA a las secuencias promotora...,3. Desestabiliza el heteroduplex DNA-RNA forma...,4. Se intercala en el molde de DNA e impide la...,1,2021
40,210. La principal fuente d e colesterol extrac...,1. Las lipoproteínas de muy baja densidad (VLDL).,2. Las lipoproteínas de baja densidad (LDL).,3. Las lipoproteínas de elevada densidad (HDL).,4. Los quilomicrones.,2,2021


In [43]:
with sqlite3.connect(db_path) as bir_warehouse:
    cur = bir_warehouse.cursor()
    for question in processed_exam_df_filtered.itertuples():
        cur.execute(query_question, (str(question[7]), exam_acronym, question[1]))
        bir_warehouse.commit()
        for n, option in enumerate(question[2:6]):
            cur.execute(query_options, (question[1], str(year), n+1, option, n+1 == question[6]))
            bir_warehouse.commit()